In [2]:
import requests
import pandas as pd
import json
from cred import credentials
from request_api import get_genesis_api
from sql import get_data

In [3]:
data = pd.read_csv('all_scaled_grouped.csv', index_col=0)
data.sort_values(by='house_price_€/m²')

,state,babyboomer_percentage,house_price_€/m²,salary,pop_diff,year,unemployment_rate,age_avg,total_population,country,...,living_space_1000qm_per_resident,scaled_number_of_buildings,scaled_living_space_1000qm,scaled_pop_diff,scaled_house_price_€/m²,scaled_salary,scaled_babyboomer_percentage,demographics,entertainment,dwelling/living_space
13,Sachsen-Anhalt,0.285721,1658,34.784,-0.151184,2020,7.7,47.536632,2180684,Germany,...,0.046686,0.678244,0.317429,1.000000,1.000000,0.084609,0.962144,0.987381,0.343131,0.658714
15,Thüringen,0.282947,1830,35.852,-0.135002,2020,6.0,47.013814,2120237,Germany,...,0.044748,0.622979,0.237901,0.920822,0.966035,0.167208,0.933139,0.921921,0.358630,0.478004
12,Sachsen,0.259114,1834,35.228,-0.067278,2020,6.1,46.367608,4056941,Germany,...,0.043189,0.444471,0.173938,0.589435,0.965245,0.118948,0.683970,0.692065,0.392879,0.558821
11,Saarland,0.276238,1935,41.313,-0.108732,2020,7.2,45.977975,983991,Germany,...,0.053374,0.854958,0.591883,0.792276,0.945300,0.589559,0.862999,0.797445,0.380875,0.502507
7,Mecklenburg-Vorpommern,0.289342,2666,33.690,-0.092362,2020,7.8,46.846078,1610774,Germany,...,0.044623,0.611567,0.232773,0.712176,0.800948,0.000000,1.000000,0.865227,0.727136,0.518531
3,Brandenburg,0.279460,2781,35.077,-0.054946,2020,6.2,46.722180,2531071,Germany,...,0.044650,0.691512,0.233879,0.529093,0.778239,0.107270,0.896677,0.762792,0.424186,0.339632
8,Niedersachsen,0.245496,2781,40.292,-0.037662,2020,5.8,44.270348,8003421,Germany,...,0.063320,1.000000,1.000000,0.444520,0.778239,0.510596,0.541594,0.478368,0.321133,0.604778
10,Rheinland-Pfalz,0.255296,2790,42.596,-0.037183,2020,5.2,44.436168,4098391,Germany,...,0.052296,0.780002,0.547655,0.442179,0.776461,0.688786,0.644052,0.521065,0.498541,0.436414
9,Nordrhein-Westfalen,0.239167,2893,43.829,-0.041035,2020,7.5,43.759486,17925570,Germany,...,0.045163,0.502697,0.254932,0.461025,0.756122,0.784145,0.475416,0.433084,0.211441,0.237464
4,Bremen,0.223559,3116,41.966,-0.034008,2020,11.2,43.143783,680130,Germany,...,0.041941,0.450114,0.122741,0.426643,0.712085,0.640062,0.312238,0.332608,0.635208,0.294483


In [16]:
columns = ['age_avg', 'employment_rate', 'new_building', 'area_new_b', 'rooms_new_b', 'cinema_seat', 'museum',
       'apartment', 'restaurant', 'living_space_1000qm', 'pop_diff', 'house_price_€/m²',	'salary', 'babyboomer_percentage']

default_weights = {'age_avg':2, 'new_building':2, 
              'area_new_b':2, 'rooms_new_b':2, 
              'cinema_seat':1, 'salary': 3,
              'museum':1,'apartment':2,
              'restaurant':1, 'employment_rate': 3,
              'living_space_1000qm': 3,
              'pop_diff': 1, 'house_price_€/m²':3, 'babyboomer_percentage': 2}

def calc_score(weights=default_weights):
    data['w_score'] = [0] * 16
    for col in columns:
        data['w_score'] += data['scaled_%s' % col] * weights[col] 
    data['avg_score'] = data['w_score'] / sum(weights.values())
    data.w_score = round(data.w_score,2)
    return data.sort_values(by='avg_score', ascending=False).reset_index()[['state','avg_score']]

In [17]:
calc_score()

,state,avg_score
0,Niedersachsen,0.615945
1,Rheinland-Pfalz,0.615641
2,Bayern,0.599694
3,Brandenburg,0.577985
4,Saarland,0.571848
5,Schleswig-Holstein,0.557729
6,Baden-Württemberg,0.552082
7,Mecklenburg-Vorpommern,0.548750
8,Sachsen-Anhalt,0.527152
9,Thüringen,0.526872


In [18]:
calc_score({'age_avg':1, 'new_building':1, 
              'area_new_b':1, 'rooms_new_b':2, 
              'cinema_seat':1, 'salary': 3,
              'museum':1,'apartment':2,
              'restaurant':1, 'employment_rate': 6,
              'living_space_1000qm': 3,
              'pop_diff': 1, 'house_price_€/m²':3, 'babyboomer_percentage': 2})

,state,avg_score
0,Rheinland-Pfalz,0.636955
1,Bayern,0.626704
2,Niedersachsen,0.626192
3,Baden-Württemberg,0.588845
4,Saarland,0.571582
5,Brandenburg,0.569411
6,Schleswig-Holstein,0.558250
7,Thüringen,0.546800
8,Sachsen,0.543073
9,Mecklenburg-Vorpommern,0.535521


In [19]:
columns = ['scaled_employment_rate' , 'scaled_house_price_€/m²', 'scaled_salary', 'demographics', 'entertainment', 'new_building', 'scaled_living_space_1000qm', 'scaled_apartment']

def calc_avg_score(weights):
    data['w_score'] = [0] * 16
    for col in columns:
        if weights[col] < 0:
            data['w_score'] += (1 - data[col]) * -weights[col]
        else:
            data['w_score'] += data[col] * weights[col]
    sum_of_weights = sum(map(abs, weights.values()))
    data['avg_score'] = data['w_score'] / sum_of_weights
    data.w_score = round(data.w_score,2)
    return data.sort_values(by='avg_score', ascending=False).reset_index()[['state','avg_score']]

In [20]:
# carlos
calc_avg_score({'scaled_employment_rate': 4, 'scaled_house_price_€/m²': 7, 'scaled_salary': 4, 'demographics': 2, 'entertainment': 3, 'new_building':  7,  'scaled_living_space_1000qm': 6, 'scaled_apartment': 5})


,state,avg_score
0,Niedersachsen,0.644218
1,Rheinland-Pfalz,0.617506
2,Saarland,0.575656
3,Bayern,0.562697
4,Sachsen,0.550568
5,Sachsen-Anhalt,0.539771
6,Brandenburg,0.535549
7,Schleswig-Holstein,0.532516
8,Baden-Württemberg,0.527929
9,Mecklenburg-Vorpommern,0.521597


In [21]:
calc_avg_score({'scaled_employment_rate': 10, 'scaled_house_price_€/m²': 2, 'scaled_salary': 10, 'demographics': -10, 'entertainment': 10, 'new_building': 7,  'scaled_living_space_1000qm': 2, 'scaled_apartment': 0})


,state,avg_score
0,Bayern,0.742369
1,Baden-Württemberg,0.741947
2,Hessen,0.646624
3,Rheinland-Pfalz,0.624961
4,Hamburg,0.615799
5,Niedersachsen,0.575991
6,Schleswig-Holstein,0.537834
7,Berlin,0.491639
8,Nordrhein-Westfalen,0.485287
9,Saarland,0.441319
